# Clause Parser Algorithm with Custom Metrics

In [1]:
!python --version
!python -m spacy download en_core_web_lg
print("Downloaded")

Python 3.6.3
    100% |████████████████████████████████| 852.3MB 20.9MB/s ta 0:00:01   14% |████▌                           | 121.3MB 3.6MB/s eta 0:03:24    25% |████████                        | 215.0MB 19.6MB/s eta 0:00:33    28% |█████████▏                      | 245.4MB 23.5MB/s eta 0:00:26    30% |█████████▊                      | 259.5MB 21.2MB/s eta 0:00:29    44% |██████████████▏                 | 378.7MB 31.6MB/s eta 0:00:16    47% |███████████████▏                | 403.1MB 68.5MB/s eta 0:00:07    50% |████████████████▎               | 433.7MB 70.5MB/s eta 0:00:06    54% |█████████████████▍              | 462.5MB 7.3MB/s eta 0:00:54    57% |██████████████████▍             | 489.9MB 72.7MB/s eta 0:00:05    58% |██████████████████▊             | 498.2MB 74.5MB/s eta 0:00:05    67% |█████████████████████▋          | 574.5MB 19.1MB/s eta 0:00:15    67% |█████████████████████▋          | 576.0MB 10.8MB/s eta 0:00:26    71% |██████████████████████▉         | 607.6MB 18.0MB/s eta 0:0

In [2]:
import spacy
import html
from spacy import displacy

nlp = spacy.load('en_core_web_lg')
print("Loaded models")

Loaded models


In [3]:
from random import shuffle
def print_sample_list(arr):
    indices = [i for i in range(len(arr))]
    shuffle(indices)
    for x in indices[:min(5, len(arr))]:
        print(x, arr[x])

In [4]:
from io import StringIO
import pandas as pd, numpy as np

tsv = StringIO("""survey_id	prompt_number	prompt_id	prompt	response	response_delimited
2546	36	48	Sometimes I wish that	my mom would come home more often.	|1| my mom |2| would come home more often.
2507	1	90	My family	has a cat...I don't have a cat though I have 2 dogs	|1| has a cat...|2| I don't have a cat |3| though I have 2 dogs
2510	33	33	When I am nervous	my hands get shaky	|1| my hands |2| get shaky
2509	22	43	At times I worry about	my own dreams, nightmares I have thousands of them	|1| my own dreams, nightmares |2| I have thousands of them
3356	34	47	Technology	its crazy and i wish i could keep up	|1| its crazy and |2| i wish |3| i could keep up
2537	26	26	When I get mad	I sometimes use big voice then I go somewhere quiet so I can cry by myself.	|1| I sometimes use big voice |2| then I go somewhere quiet |3| so I can cry by myself.
1869	25	25	My main problem is	a feeling that I am not entitled	|1| a feeling |2| that I am not entitled
3127	29	29	If my mother	was a different lady, I would not be who I am.	|1| was a different lady, |2| I would not be who I am.
1832	26	26	When I get mad	it takes a long time to build up, but I am slow to forgive.	|1| it takes a long time to build up, |2| but I am slow to forgive.
2390	17	17	When they avoided me	I felt safe, knowing that I could receive a warm welcome from my family and friends.	|1| I felt safe, |2| knowing |3| that I could receive a warm welcome |4| from my family and friends.
1880	36	48	Sometimes I wish that	I could be younger and have done things differently.	|1| I could be younger |2| and have done things differently.
2407	17	17	When they avoided me	I sought to find out 'why?' . Knowing the 'why' can begin Awareness and Healing	|1| I sought |2| to find out 'why?' . |3| Knowing the 'why' |4| can begin Awareness and Healing
3233	5	5	Being with other people	is both a learning experience and an opportunity to exchange views.	|1| is both a learning experience and an opportunity |2| to exchange views.
2102	25	25	My main problem is	only a main problem if I think it is.	|1| only a main problem |2| if I think |3| it is.
1529	10	10	When people are helpless	At times I try to find other ways of doing thing. But sometime I might be disappointed and stop or quit.	|1| At times I try |2| to find other ways |3| of doing thing. |4| But sometime I might be disappointed |5| and stop |6| or quit.
2856	31	31	My father	taught me how to think critically and to love deeply.	|1| taught me |2| how to think critically |3| and to love deeply.
1975	32	32	If I can\'t get what I want	I try to work out another way of getting what I want	|1| I try |2| to work out another way |3| of getting |4| what I want
2157	17	17	When they avoided me	hmmm... I\'m stumped. I have no idea how to respond to this... Although there is a sense of relief...	|1| hmmm... I'm stumped. |2| I have no idea |3| how to respond to this...|4| Although there is a sense of relief...
2409	5	73	Loving other people	can deepen with familiarity. Even with those who aren't a natural fit the differences between us can add color to the world as well as an opportunity to turn inward to see what is being triggered. Having said that, it's difficult for me to open my heart and trust.	|1| can deepen with familiarity. |2| Even with those who aren't a natural fit |3| the differences between us can add color to the world |4| as well as an opportunity to turn inward |5| to see |6| what is being triggered. |7| Having said that, |8| it's difficult for me |9| to open |10| my heart |11| and trust.
2046	10	10	When people are helpless	I am inspired to reach out and be of service.	|1| I am inspired |2| to reach out |3| and be of service.
2837	18	42	Rules	are guard rails. It is good and helpful to have them. When you head in the right direction you never get in touch with them, no matter how fast or slow you travel. Good rules are not in your way.	|1| are guard rails. |2| It is good and helpful |3| to have them. |4| When you head in the right direction |5| you never get in touch with them, |6| no matter how fast or slow you travel. |7| Good rules are not in |8| your way.
2381	34	47	Technology	is a great tool, nice to have and can bring global humanity closer together in ways never before imagined. But if it were to disappear tomorrow, I'm of the generation that would be able to survive.	|1| is a great tool, |2| nice to have |3| and can bring global humanity closer together |4| in ways never before imagined. |5| But if it were |6| to disappear tomorrow, |7| I'm of the generation |8| that would be able |9| to survive.
2344	12	12	A good boss	A good boss is critical toward making a workplace environment healthy and successful!	|1| A good boss is critical |2| toward making a workplace environment healthy and successful!
2952	25	25	My main problem is	getting over decades spent proving myself.	|1| getting over |2| decades spent proving myself.
2721	16	16	I feel sorry	for the pain that all humans and all beings on this earth must endure as part of the experience of being here.	|1| for the pain |2| that all humans and all beings on this earth must endure |3| as part of the experience of being here.
2276	30	30	If I were in charge	i would change that by facilitating change that made everyone in charge ... to the degree of their capabilities in wordly matters, but always in charge of their interior and their right to decide for themselves what is real and true for them.	|1| i would change that |2| by facilitating change |3| that made everyone in charge ...|4| to the degree of their capabilities in wordly matters,|5| but always in charge |6| of their interior and |7| their right |8| to decide for themselves |9| what is real and true for them.
3099	22	43	At times I worry about	big, big existential questions, and little microscopic ones... sometimes within milliseconds of one another.	|1| big, big existential questions, and little microscopic ones...|2| sometimes within milliseconds of one another.
1883	1	1	Raising a family	is a profound gift. I am tending the future. I am tending the life processes. It is a place for love and life and evolution to soar.	|1| is a profound gift. |2| I am tending the future. |3| I am tending the life processes. |4| It is a place for love and life and evolution |5| to soar.
1996	22	43	At times I worry about	how differently I see and process things from many of the people I encounter, and whether I am wisely acting and interacting.	|1| how differently I see |2| and process things |3| from many of the people I encounter, |4| and whether I am wisely acting |5| and interacting.
2374	11	39	What I like to do best is	be a conduit. There is no I.	|1| be a conduit. |2| There is no I.
3158	5	5	Being with other people	can be an isolating or a hugely enriching experience, the latter when each others presence liberates emergent thoughts or self expressions bringing forth a collective that is more than the sum of its parts	|1| can be an isolating or a hugely enriching experience, |1| the latter when each others presence liberates emergent thoughts or self expressions |3| bringing forth a collective |4| that is more than the sum of its parts
2690	28	83	A teacher has the right to	I don't mean to be oppositional. . but this sentence stem. . .like so many others. . isn't one I can really answer outside of context. Rights. This is a wobbly construct. I'm sorry. I'm out of time otherwise I would write a bit about the possibility of rights . .the idea of having a right. . the usage of the language of rights.	|1| I don't mean |2| to be oppositional. .|3| but this sentence stem. . .like so many others. . isn't one |4| I can really answer outside of context.|5| Rights. This is a wobbly construct.|6| I'm sorry.|7| I'm out of time |8| otherwise I would write a bit about the possibility of rights ..the idea of having a right. . the usage of the language of rights.
3321	5	5	Being with other people	is most often a pleasure and fascinating experience of seeing a completely different version of Self.	|1| is most often a pleasure and fascinating experience |2| of seeing a completely different version of Self.
2404	14	14	The past	is a construct to give form and structure to the present moment.	|1| is a construct |2| to give form and structure to the present moment.
2215	13	40	We could make the world a better place if	the mystery was okay to be as a question, that is to say if the question was okay to be a question and the answer could be a question and the answers and questions could be held together because they need each other and we realize a little faster that they won't end and they are growing together and rest in that.	|1| the mystery was okay |2| to be as a question, |3| that is to say |4| if the question was okay |5| to be a question |6| and the answer could be a question |7| and the answers and questions could be held together |8| because they need each other |9| and we realize a little faster |10| that they won't end |11| and they are growing together |12| and rest in that.
3181	12	12	A good boss	is a solar being ... one that brings light and eternal love without demanding anything of the others ... an energetic and gravitational field of such calm, peaceful magnitude that all are warmed and captivated by their glow ... brings a new burst life to all those who are in their presence	|1| is a solar being ...|2| one that brings light and eternal love |3| without demanding anything of the others ..|4| an energetic and gravitational field of such calm, peaceful magnitude |5| that all are warmed and captivated by their glow ... |6| brings a new burst life |7| to all those who are in their presence
1823	35	35	My conscience bothers me if	I'm not placing my feet on the path with an open heart, mind and spirit, tuning in to what's moving and showing up fully at the dance.	|1| I'm not placing |2| my feet on the path with an open heart, mind and spirit,|3| tuning in to |4| what's moving |5| and showing up fully at the dance.
2619	2	2	When I am criticized	I can embody my personality and feel a pang contrast to the story I tell myself inside the story we tell ourselves of what we think in the moment to be an absolute before it falls away into a deeper, clearer calm of letting go.	|1| I can embody |2| my personality |3| and feel a pang contrast |4| to the story I tell myself |5| inside the story we tell ourselves |6| of what we think in the moment |7| to be an absolute |8| before it falls away into a deeper, clearer calm |9| of letting go.
1823	6	6	The thing I like about myself is	that here I sit in the drop of this essential moment, chuckling at this question! :-D	|1| that here I sit in the drop of this essential moment, |2| chuckling at this question! :-D
1909	25	25	My main problem is	integrating emptiness with everyday life, finding meaning in meaningless given the meaningless in meaning making.	|1| integrating emptiness with everyday life, |2| finding meaning in meaningless |3| given the meaningless |4| in meaning making.
1814	25	25	My main problem is	how my narccissim gets in the way of seeing the truth of the illusion of self.	|1| how my narcissm |2| gets in the way |3| of seeing the truth of the illusion of self
1805	4	37	These days, work	Is a rainbow of consciousness manifesting through each sparkling emanation called a 'me'.	|1| Is a rainbow of consciousness |2| manifesting through each sparkling emanation |3| called a 'me'.
1665	5	5	Being with other people	Is a mutually mutating meeting of universes	|1| Is a mutually mutating meeting of universes
2182	11	39	What I like to do best is	lay back and live the window that I have becomeas if a home with no walls, the infinite moves through as if there is no solid boundary.the veil is no more and the whole field that is everywhere exists where the inside and outside have no difference.	|1| lay back |2| and live the window |3| that I have become as if a home with no walls, |4| the infinite moves through |5| as if there is no solid boundary. |6| the veil is no more |7| and the whole field that is everywhere |8| exists where the inside and outside |9| have no difference.
1823	3	3	Change is	the heartbeat of the universe... aliveness calling us forth!	|1| the heartbeat of the universe...|2| aliveness calling us forth!
2721	26	26	When I get mad	i feel adrenaline through my human body and it is fierce and i love it because it moves me and it is no different than the Spirit which moves all.	|1| i feel adrenaline |2| through my human body |3| and it is fierce |4| and i love it |5| because it moves me |6| and it is no different |7| than the Spirit which moves all.
2338	1	1	Raising a family	Captivates my whole being, just as every window on reality sees it is held in Love beyond knowing.	|1| Captivates |2| my whole being, |3| just as every window on reality sees |4| it is held in Love beyond knowing.
2721	30	30	If I were in charge	Is an expression of the love animating all creation, the simple joy that moves all from rock to life to mind to All. It comes through this glorious One in all its parts and territories, from an inaccessible Heart, within, without.	|1| Is an expression |2| of the love animating all creation, |3| the simple joy that moves all from rock to life to mind to All. |4| It comes through this glorious One in all its parts and territories, from an inaccessible Heart, within, without.
1889	15	41	Privacy	the human formconsicousness separates from the One even for a moment.	|1| the human form |2| consicousness separates from the One even for a moment.""")
df = pd.read_csv(tsv, delimiter = "\t")
print(df.columns)
df.sample(frac=1).head()

Index(['survey_id', 'prompt_number', 'prompt_id', 'prompt', 'response',
       'response_delimited'],
      dtype='object')


,survey_id,prompt_number,prompt_id,prompt,response,response_delimited
45,2721,26,26,When I get mad,i feel adrenaline through my human body and it...,|1| i feel adrenaline |2| through my human bod...
1,2507,1,90,My family,has a cat...I don't have a cat though I have 2...,|1| has a cat...|2| I don't have a cat |3| tho...
22,2344,12,12,A good boss,A good boss is critical toward making a workpl...,|1| A good boss is critical |2| toward making ...
18,2409,5,73,Loving other people,can deepen with familiarity. Even with those w...,|1| can deepen with familiarity. |2| Even with...
44,1823,3,3,Change is,the heartbeat of the universe... aliveness cal...,|1| the heartbeat of the universe...|2| aliven...


In [5]:
if "prompt" in df.columns: #Original dataset
    df['sentence'] = df.apply(lambda row : "{} {}".format(row['prompt'], row['response']), axis = 1)

df.sample(frac=1).head()

,survey_id,prompt_number,prompt_id,prompt,response,response_delimited,sentence
16,1975,32,32,If I can't get what I want,I try to work out another way of getting what ...,|1| I try |2| to work out another way |3| of g...,If I can't get what I want I try to work out a...
3,2509,22,43,At times I worry about,"my own dreams, nightmares I have thousands of ...","|1| my own dreams, nightmares |2| I have thous...","At times I worry about my own dreams, nightmar..."
43,2182,11,39,What I like to do best is,lay back and live the window that I have becom...,|1| lay back |2| and live the window |3| that ...,What I like to do best is lay back and live th...
12,3233,5,5,Being with other people,is both a learning experience and an opportuni...,|1| is both a learning experience and an oppor...,Being with other people is both a learning exp...
42,1665,5,5,Being with other people,Is a mutually mutating meeting of universes,|1| Is a mutually mutating meeting of universes,Being with other people Is a mutually mutating...


In [6]:
import re, html
PATTERN = "[^a-zA-Z0-9\s]+"
rgx = re.compile(PATTERN, re.IGNORECASE)

df['preprocessed_sentence'] = df['sentence'].apply(lambda ip : re.sub('\s+', ' ', rgx.sub(' ', html.unescape(ip))))
print(df.columns)
df.sample(frac=1).head()

Index(['survey_id', 'prompt_number', 'prompt_id', 'prompt', 'response',
       'response_delimited', 'sentence', 'preprocessed_sentence'],
      dtype='object')


,survey_id,prompt_number,prompt_id,prompt,response,response_delimited,sentence,preprocessed_sentence
27,1883,1,1,Raising a family,is a profound gift. I am tending the future. I...,|1| is a profound gift. |2| I am tending the f...,Raising a family is a profound gift. I am tend...,Raising a family is a profound gift I am tendi...
9,2390,17,17,When they avoided me,"I felt safe, knowing that I could receive a wa...","|1| I felt safe, |2| knowing |3| that I could ...","When they avoided me I felt safe, knowing that...",When they avoided me I felt safe knowing that ...
34,2215,13,40,We could make the world a better place if,"the mystery was okay to be as a question, that...",|1| the mystery was okay |2| to be as a questi...,We could make the world a better place if the ...,We could make the world a better place if the ...
5,2537,26,26,When I get mad,I sometimes use big voice then I go somewhere ...,|1| I sometimes use big voice |2| then I go so...,When I get mad I sometimes use big voice then ...,When I get mad I sometimes use big voice then ...
38,1823,6,6,The thing I like about myself is,that here I sit in the drop of this essential ...,|1| that here I sit in the drop of this essent...,The thing I like about myself is that here I s...,The thing I like about myself is that here I s...


In [7]:
df['nlp_doc'] = df['preprocessed_sentence'].apply(lambda ip : nlp(ip))
print(df.columns)
df.sample(frac=1).head()

Index(['survey_id', 'prompt_number', 'prompt_id', 'prompt', 'response',
       'response_delimited', 'sentence', 'preprocessed_sentence', 'nlp_doc'],
      dtype='object')


,survey_id,prompt_number,prompt_id,prompt,response,response_delimited,sentence,preprocessed_sentence,nlp_doc
10,1880,36,48,Sometimes I wish that,I could be younger and have done things differ...,|1| I could be younger |2| and have done thing...,Sometimes I wish that I could be younger and h...,Sometimes I wish that I could be younger and h...,"(Sometimes, I, wish, that, I, could, be, young..."
23,2952,25,25,My main problem is,getting over decades spent proving myself.,|1| getting over |2| decades spent proving mys...,My main problem is getting over decades spent ...,My main problem is getting over decades spent ...,"(My, main, problem, is, getting, over, decades..."
46,2338,1,1,Raising a family,"Captivates my whole being, just as every windo...","|1| Captivates |2| my whole being, |3| just as...","Raising a family Captivates my whole being, ju...",Raising a family Captivates my whole being jus...,"(Raising, a, family, Captivates, my, whole, be..."
39,1909,25,25,My main problem is,"integrating emptiness with everyday life, find...","|1| integrating emptiness with everyday life, ...",My main problem is integrating emptiness with ...,My main problem is integrating emptiness with ...,"(My, main, problem, is, integrating, emptiness..."
9,2390,17,17,When they avoided me,"I felt safe, knowing that I could receive a wa...","|1| I felt safe, |2| knowing |3| that I could ...","When they avoided me I felt safe, knowing that...",When they avoided me I felt safe knowing that ...,"(When, they, avoided, me, I, felt, safe, knowi..."


#### Metrics

* Total % of sentences with correct reconstructions =  0.9061 . It's actually greater than 91% since complex first clauses followed by conjunctions put the conjuction with the parent clause in the first.
* Response expected = actual verbatim : 

In [8]:
def flatten_list(l):
    flat_list = [item for sublist in l for item in sublist]
    return flat_list

def get_children(doc):
    if len([x for x in doc.children]) == 0:
        return [doc]
    if doc.pos_ == "VERB" and doc.dep_ not in ["xcomp", "aux"]:
        return []

    op = flatten_list([get_children(l) for l in doc.lefts]) + [doc] + flatten_list([get_children(r) for r in doc.rights])
    return op

def postprocess(tokens_arr):
    if len(tokens_arr) == 1 and ( tokens_arr[0].dep_ in ["aux", "auxpass"] or tokens_arr[0].tag_ in ["VBG"]): 
        return []
    return tokens_arr

def get_text_from_tokens(tokens_arr):
    op = ' '.join([x.text for x in tokens_arr])
    op = op.replace(" nt", "nt").replace(" '", "'")
    return op

def clause_split_by_verbs(doc):
    op = []
    for token in doc:
        if token.pos_ == "VERB":
            arr = flatten_list([get_children(l) for l in token.lefts]) + [token] + flatten_list([get_children(r) for r in token.rights])
            arr = postprocess(arr)
            op.append(arr)
    if len(op)==0:
        op.append(doc)
    return op

df['split_by_verbs_arr'] = df['nlp_doc'].apply(clause_split_by_verbs)
df.sample(frac = 1).head()

,survey_id,prompt_number,prompt_id,prompt,response,response_delimited,sentence,preprocessed_sentence,nlp_doc,split_by_verbs_arr
1,2507,1,90,My family,has a cat...I don't have a cat though I have 2...,|1| has a cat...|2| I don't have a cat |3| tho...,My family has a cat...I don't have a cat thoug...,My family has a cat I don t have a cat though ...,"(My, family, has, a, cat, I, don, t, have, a, ...","[[My, family, has, a, cat], [I, don, t], [have..."
24,2721,16,16,I feel sorry,for the pain that all humans and all beings on...,|1| for the pain |2| that all humans and all b...,I feel sorry for the pain that all humans and ...,I feel sorry for the pain that all humans and ...,"(I, feel, sorry, for, the, pain, that, all, hu...","[[I, feel, sorry, for, the, pain], [], [that, ..."
5,2537,26,26,When I get mad,I sometimes use big voice then I go somewhere ...,|1| I sometimes use big voice |2| then I go so...,When I get mad I sometimes use big voice then ...,When I get mad I sometimes use big voice then ...,"(When, I, get, mad, I, sometimes, use, big, vo...","[[When, I, get, mad], [I, sometimes, use, big,..."
16,1975,32,32,If I can't get what I want,I try to work out another way of getting what ...,|1| I try |2| to work out another way |3| of g...,If I can't get what I want I try to work out a...,If I can t get what I want I try to work out a...,"(If, I, can, t, get, what, I, want, I, try, to...","[[], [If, I, can, t], [get], [what, I, want], ..."
36,1823,35,35,My conscience bothers me if,I'm not placing my feet on the path with an op...,|1| I'm not placing |2| my feet on the path wi...,My conscience bothers me if I'm not placing my...,My conscience bothers me if I m not placing my...,"(My, conscience, bothers, me, if, I, m, not, p...","[[My, conscience, bothers, me, not, placing, m..."


df postprocessing and the clause delimiting

In [9]:
df['split_by_verbs'] = df['split_by_verbs_arr'].apply(lambda op : ' '.join(["|{}| {}".format(i+1, op[i]) for i in range(len(op))]))

def remove_prompts(df):
    prompt, tokens_arr = df.prompt, df.split_by_verbs_arr
    pdoc = nlp(prompt)
    ignore_indices = [x.i for x in pdoc]
    new_arr = []
    for clause in tokens_arr:
        new_clause = [t for t in clause if t.i not in ignore_indices]
        if len(new_clause) >= 0:
            new_arr.append(new_clause)
    return new_arr

def process_text_df(clauses_arr):
    new_arr = []
    # first pass
    first_pass = []
    tok_arr = [[ tok.i for tok in clause] for clause in clauses_arr]

    for i in range(len(tok_arr)):
        x = tok_arr[i]
        if len(x) ==  0:
            continue
        is_subset = False
        for y in tok_arr:
            if set(x).issubset(y) and not set(x) == set(y):
                is_subset = True
        if not is_subset:
            first_pass.append(clauses_arr[i])
    
    for clauses in first_pass:
        if len(clauses) == 0:
            continue
        txt = get_text_from_tokens(clauses)
        new_arr.append(txt)
    
    return new_arr
        
ignore_df = None
if "prompt" in df.columns:
    df['delltr'] = df[['prompt', 'split_by_verbs_arr']].apply(remove_prompts, axis = 1) 
    df['split_by_verbs_arr_no_prompt'] = df['delltr'].apply(process_text_df)
    del df['delltr']
    df['split_by_verbs'] = df['split_by_verbs_arr_no_prompt'].apply(lambda op : ' '.join(["|{}| {}".format(i+1, op[i]) for i in range(len(op))]))
    df['is_correct'] = df.apply(lambda x : re.sub("[a-z0-9]+", "", x['response_delimited'].strip().lower()) == re.sub("[a-z0-9]+", "", x['split_by_verbs'].strip().lower()), axis = 1)
    ignore_df = df[['response_delimited', 'split_by_verbs', 'is_correct']].sample(frac = 1).head()
else:
    df['split_by_verbs_arr_cleaned'] = df['split_by_verbs_arr'].apply(process_text_df)
    df['split_by_verbs'] = df['split_by_verbs_arr_cleaned'].apply(lambda op : ' '.join(["||{}| {}".format(i+1, op[i]) for i in range(len(op))]))
    df['tokens_set'] = df['nlp_doc'].apply(lambda doc : ";".join(["{}={}, {}, {}".format(token.text, token.tag_, token.dep_, token.pos_) for token in doc]))
    df[['tempID', 'sentence', 'split_by_verbs', 'tokens_set']].to_csv("./output.csv", index = False)
    df['reconstructed'] = df['split_by_verbs'].apply(lambda x : re.sub("\s+", " ", re.sub("\|\|\d+\|", "", x)))    
    df['is_correct'] = df.apply(lambda x : re.sub("[a-z0-9]+", "", x['preprocessed_sentence'].strip().lower()) == re.sub("[a-zA-Z0-9]+", "", x['reconstructed'].strip().lower()), axis = 1)
    ignore_df = df[['preprocessed_sentence', 'reconstructed', 'is_correct']].sample(frac=1).head()

print(df.columns)
ignore_df

Index(['survey_id', 'prompt_number', 'prompt_id', 'prompt', 'response',
       'response_delimited', 'sentence', 'preprocessed_sentence', 'nlp_doc',
       'split_by_verbs_arr', 'split_by_verbs', 'split_by_verbs_arr_no_prompt',
       'is_correct'],
      dtype='object')


,response_delimited,split_by_verbs,is_correct
25,|1| i would change that |2| by facilitating ch...,|1| i would change that by |2| facilitating ch...,False
28,|1| how differently I see |2| and process thin...,|1| I see and and |2| how differently process ...,False
45,|1| i feel adrenaline |2| through my human bod...,|1| i feel adrenaline through my human body an...,False
6,|1| a feeling |2| that I am not entitled,|1| a feeling |2| that I am not entitled,True
11,|1| I sought |2| to find out 'why?' . |3| Know...,|1| I sought to find out |2| why Knowing |3| t...,False


In [10]:
df[['sentence', 'split_by_verbs']].sample(frac = 10/df.shape[0])

,sentence,split_by_verbs
20,Rules are guard rails. It is good and helpful ...,|1| are guard rails |2| It is good and helpful...
4,Technology its crazy and i wish i could keep up,|1| and i wish |2| i could keep up
19,When people are helpless I am inspired to reac...,|1| I am inspired to reach out and |2| be of s...
22,A good boss A good boss is critical toward mak...,|1| A good boss is critical toward |2| making ...
45,When I get mad i feel adrenaline through my hu...,|1| i feel adrenaline through my human body an...
5,When I get mad I sometimes use big voice then ...,|1| I sometimes use big voice |2| then I go so...
14,When people are helpless At times I try to fin...,|1| At times |2| I try to find other ways of |...
47,If I were in charge Is an expression of the lo...,|1| Is an expression of the love |2| animating...
1,My family has a cat...I don't have a cat thoug...,|1| has a cat |2| I don t |3| have a cat |4| t...
25,If I were in charge i would change that by fac...,|1| i would change that by |2| facilitating ch...


In [11]:
print("Ignorable metric sentence reformation", df[df.is_correct].shape[0]/df.shape[0])

Ignorable metric sentence reformation 0.02040816326530612
